# Equity Analyst Agent with Assistants API and Function calling

### This notebook showcases the capabilities of Azure OpenAI's Assistants API for an Equity Analyst Agent. 

Install the necessary Python packages (openai, matplotlib, tenacity, python-dotenv) for the notebook to function.

In [ ]:
%pip install openai matplotlib tenacity python-dotenv

Imports the necessary Python modules and classes used in the notebook. Note the openai module is used to interact with the Assistants API.

In [ ]:
import os
import html
from pathlib import Path
from dotenv import load_dotenv
from openai import AzureOpenAI
from tenacity import RetryError

from azure_ai_util import AzureAIUtils, NotCompletedException

### Initialize Azure OpenAI Client
This cell crucial for establishing communication with Azure OpenAI services. 

- **Requirement**: Ensure that a `.env` file exists in the same directory as this notebook. This file should contain the necessary API credentials and configuration details, which you can obtain from https://ai.azure.com 

In [ ]:
load_dotenv(".env")
client = AzureOpenAI(api_key=os.getenv("OPENAI_API_KEY"), 
                     azure_endpoint=os.getenv("OPENAI_ENDPOINT"),
                     api_version="2024-02-15-preview")

utils = AzureAIUtils(client)

### Create the assistant with tools and files


The `create_assistant` function creates an assistant with tools and files. The function takes the following parameters:
- `name`: The name of the assistant.
- `instructions`: The system message (or meta prompt) that gives the assistant a persona and context.
- `tools`: A list of tools that the assistant can use to perform tasks. Currently, these are `code_intrepreter` and `retriever`.
- `functions`: Custom functions that the assistant can use to perform tasks. Similar to function calling feature.
- `model`: The name of the model to use for the assistant.

In [ ]:
DATA_COLLECTION_LOGIC_APPS_URI = os.getenv("DATA_COLLECTION_LOGIC_APPS_URI")

def get_proprietary_data(ticker: str, data_type: str, email: str) -> None:
    json_payload = {"ticker": ticker, "data_type": data_type, "email": email}
    headers = {"Content-Type": "application/json"}
    response = requests.post(DATA_COLLECTION_LOGIC_APPS_URI, json=json_payload, headers=headers)
    if response.status_code == 202:
        print("Email sent to: " + json_payload["to"])

def get_current_share_price(ticker: str) -> float:
    stock = yf.Ticker(ticker)
    return stock.history(period="1d")["Close"].iloc[-1]

In [ ]:
# Fetch the files under the datasets directory
DATASETS = "datasets/"

assistant_files = [utils.upload_file(Path(DATASETS) / file) for file in os.listdir(DATASETS)]
file_ids = [file.id for file in assistant_files]

Create the assistant with tools and files

In [ ]:
tools_list = [
    {"type": "code_interpreter"},
    {
        "type": "function",
        "function": {
            "name": "get_proprietary_data",
            "description": "Gets proprietary data from a Logic Apps workflow and notifies the user via email.",
            "parameters": {
                "type": "object",
                "properties": {
                    "ticker": {"type": "string", "description": "The stock ticker symbol."},
                    "data_type": {"type": "string", "description": "The type of data to retrieve. ARPU, EPS, Margin, etc."},
                    "email": {"type": "string", "description": "The email address to notify."}
                },
                "required": ["to", "content"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "get_current_share_price",
            "description": "Get the current share price of a stock or symbol.",
            "parameters": {
                "type": "object",
                "properties": {"ticker": {"type": "string", "description": "The ticker of the stock"}},
                "required": ["ticker"],
            },
        },
    }
]

In [ ]:
def call_functions(client: AzureOpenAI, thread: Thread, run: Run) -> None:
    print("Function Calling")
    required_actions = run.required_action.submit_tool_outputs.model_dump()
    print(required_actions)
    tool_outputs = []
    import json

    for action in required_actions["tool_calls"]:
        func_name = action["function"]["name"]
        arguments = json.loads(action["function"]["arguments"])

        if func_name == "get_stock_price":
            output = get_current_share_price(ticker=arguments["ticker"])
            tool_outputs.append({"tool_call_id": action["id"], "output": output})
        elif func_name == "send_email":
            print("Fetching proprietary data")
            ticker = arguments["ticker"]
            data_type = arguments["data_type"]
            email = arguments["email"]
            get_proprietary_data(ticker, data_type, email)

            tool_outputs.append({"tool_call_id": action["id"], "output": "Fetched proprietary data and notified user."})
        else:
            raise ValueError(f"Unknown function: {func_name}")

    print("Sending tool outputs to the thread.")
    client.beta.threads.runs.submit_tool_outputs(thread_id=thread.id, run_id=run.id, tool_outputs=tool_outputs)

In [ ]:
assistant = client.beta.assistants.create(
    name="Equity Analyst",
    instructions=("You are an equity analyst that performs analysis on the given datasets. "
                  "Use the given tools to help you gather data and perform analysis."
                  "With your tools, you can retrieve the latest stock price, fetch proprietary data, and notify user."),
    tools=tools_list,
    file_ids=file_ids,
    model=os.getenv("OPENAI_MODEL_NAME")
)

Create a thread, which represents a conversation. It is recommended to create one thread per user. 

In [ ]:
thread = client.beta.threads.create()

Create a thread run

In [ ]:
def analyst_assistant(content: str):
    client.beta.threads.messages.create(thread_id=thread.id, role="user", content=content)

    run = client.beta.threads.runs.create(
        thread_id=thread.id,
        assistant_id=assistant.id,
        instructions=f"You are a equity analyst who maps out the ask of the user to an equity analyst's task and thinks step by step to analyze, including making use of the tools. Make generic assumptions",
    )

    try:
        run = utils.get_run_lifecycle_status(thread.id, run.id)
        messages = client.beta.threads.messages.list(thread_id=thread.id)
        utils.format_response(messages)
    except RetryError:
        print("Operation failed or timed out after maximum retries.")
    except NotCompletedException:
        print("Operation did not complete in the expected status.")


### Have the assistant perform a DCF valuation

In [ ]:
analyst_assistant("Visualize the data and provide insights on the trends.")

In [ ]:
analyst_assistant("Perform a discounted cash flow valuation using the provided dataset, and print the code along with its executed output that was used for this calculation.")

In [ ]:
analyst_assistant("Summarize and Visualize this information to someone new to finance and investing.")

### Delete the thread and assistant

In [ ]:
for entity in [(client.beta.assistants, assistant), (client.beta.threads, thread)]:
    entity[0].delete(entity[1].id)

for file in assistant_files:
    client.files.delete(file.id)